In [1]:
import os
import sys
import pathlib
import typing

import numpy as np

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [11]:
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, GlobalAveragePooling2D
from keras.layers import Dropout, BatchNormalization
from keras.callbacks import Callback, ModelCheckpoint
from keras.optimizers import adam, SGD

import ml.generators.mip_generator as generator

import tensorflow as tf

In [12]:
num_filts, filt_dims = 96, 3
channels, im_dims = 3, 120
output_dim = 1

epochs = 20
batch_size = 16

In [13]:
train_gen = generator.MipGenerator(dims=(channels, im_dims, im_dims),
                          batch_size=batch_size,
                          augment_data=True,
                          extend_dims=False,
                          validation=False)
val_gen = generator.MipGenerator(dims=(channels, im_dims, im_dims),
                          batch_size=batch_size,
                          augment_data=True,
                          extend_dims=False,
                          validation=True)

In [14]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [15]:
model = Sequential()
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first',
                 activation='relu', padding='same', input_shape=(channels, im_dims, im_dims)))
model.add(BatchNormalization())
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same', strides=(2,2)))
model.add(BatchNormalization())
num_filts *= 2
model.add(Dropout(0.4))
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same', strides=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(num_filts, (filt_dims, filt_dims), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(num_filts, (1, 1), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(1, (1, 1), data_format='channels_first', activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D(data_format='channels_first'))
model.add(Dense(output_dim, activation='sigmoid'))
model.compile(optimizer=adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 96, 120, 120)      2688      
_________________________________________________________________
batch_normalization_10 (Batc (None, 96, 120, 120)      480       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 96, 120, 120)      83040     
_________________________________________________________________
batch_normalization_11 (Batc (None, 96, 120, 120)      480       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 96, 60, 60)        83040     
_________________________________________________________________
batch_normalization_12 (Batc (None, 96, 60, 60)        240       
_________________________________________________________________
dropout_3 (Dropout)          (None, 96, 60, 60)        0         
__________

In [18]:
mc_callback = ModelCheckpoint(filepath='tmp/all_conv_weights.hdf5', verbose=1)
model.fit_generator(
    generator=train_gen.generate(),
    steps_per_epoch=train_gen.get_steps_per_epoch(),
    validation_data=val_gen.generate(),
    validation_steps=val_gen.get_steps_per_epoch(),
    epochs=epochs,
    callbacks=[mc_callback],
    verbose=1,
    max_queue_size=1)

0Epoch 1/20



/home/harold_triedman/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Loaded entire batch.
(16, 3, 120, 3)


ValueError: Error when checking input: expected conv2d_10_input to have shape (3, 120, 120) but got array with shape (3, 120, 3)